In [1]:
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from sklearn.model_selection import train_test_split
import catboost as cb
from sklearn.metrics import classification_report

In [2]:
file = "type_contract.csv"
contracts = pd.read_csv(file)
contracts
#сначала читаем этот файл, т.к. предполагаем, что в нем есть наиболее 
#полный список contract_id (всех клиентов, которые у нас есть)

,contract_id,day_or_month_contract
0,7780,0
1,3996,0
2,7785,0
3,7794,0
4,7795,1
...,...,...
6800,3503,1
6801,3589,0
6802,4261,0
6803,4118,1


In [3]:
contracts['contract_id'].nunique()
#есть дубликаты, т.к. из 6805 строк уникальных 6800

6800

In [4]:
contracts[contracts.duplicated(['contract_id'])]
#давайте посмотрим на них

,contract_id,day_or_month_contract
134,3914,0
598,3859,0
599,3859,0
600,3859,0
3200,38532,0


In [5]:
contracts = contracts.drop_duplicates(subset=['contract_id'])
contracts
#теперь без повторов, у нас выборка из 6800 клиентов

,contract_id,day_or_month_contract
0,7780,0
1,3996,0
2,7785,0
3,7794,0
4,7795,1
...,...,...
6800,3503,1
6801,3589,0
6802,4261,0
6803,4118,1


In [6]:
file = "train.csv"
train_data = pd.read_csv(file)
train_data
#посмотрим, из чего состоит наша тренировочная выборка

,contract_id,blocked
0,7780,0
1,7785,0
2,7794,0
3,7795,0
4,7798,0
...,...,...
5987,274601,1
5988,274710,1
5989,274782,1
5990,274786,1


In [7]:
train_data['contract_id'].nunique()
#дубликатов нет, значит нам останется выборка из 6800-5992=808 клиентов для предсказывания оттока

5992

In [8]:
file = "log.csv"
logs = pd.read_csv(file)
logs

,contract_id,event_date,event_type
0,36294,2021-03-28 16:24:30,Добавление в Обращались с номеров
1,36294,2021-03-28 16:27:41,Обращение в службу заботы о клиентах
2,36294,2021-03-28 16:29:56,Выключение IPTV-пакета
3,36294,2021-03-28 16:29:56,Включение IPTV-пакета
4,36294,2021-03-28 16:35:04,Обращение в службу заботы о клиентах
...,...,...,...
25269,11801,2021-04-29 07:50:44,Информер ВК. Показ
25270,54405,2021-03-17 10:38:42,Информер ВК. Показ
25271,54405,2021-03-23 08:21:29,Информер ВК. Показ
25272,54405,2021-04-09 13:04:26,Информер ВК. Показ


In [9]:
logs['event_type'].nunique()
#105 уникальных причин обращения

105

In [10]:
logs['number_support'] = logs.groupby(logs['contract_id'])['contract_id'].transform('size')
#К сожалению, я пока не знаю, как правильно работать с данными об истории обращений, запросов и т.д.
#Поэтому я делаю предположение, что важны следующие параметры: общее число обращений и причина последнего обращения

In [11]:
logs.info()
#видим, что вторая колонка у нас не время, а должна быть им

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25274 entries, 0 to 25273
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   contract_id     25274 non-null  int64 
 1   event_date      25274 non-null  object
 2   event_type      25274 non-null  object
 3   number_support  25274 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 789.9+ KB


In [12]:
logs['event_date'] = pd.to_datetime(logs['event_date'])

In [13]:
logs = logs.sort_values('event_date').drop_duplicates('contract_id', keep='last')
logs
#оставляем только последнюю причину обращения

,contract_id,event_date,event_type,number_support
8321,195133,2021-03-01 00:02:01,Включение интернета на 20 минут,1
17030,46382,2021-03-01 01:54:24,Информер ВК. Показ,1
15926,14970,2021-03-01 08:30:52,Информер ВК. Показ,1
16802,46120,2021-03-01 08:49:46,Гарантированный платеж за деньги,1
16072,12496,2021-03-01 10:51:46,Информер ВК. Показ,1
...,...,...,...,...
23697,53432,2021-04-29 23:04:39,Гарантированный платеж за деньги,8
20637,50543,2021-04-29 23:05:10,Включение интернета на 20 минут,9
19871,49920,2021-04-29 23:22:46,Информер ВК. Показ,8
21224,49469,2021-04-29 23:40:20,Информер ВК. Показ,7


In [14]:
file = "named.csv"
names = pd.read_csv(file)
names

,date,url,contract_id
0,17-Apr-2021,webmail.sampo.ru,101397
1,17-Apr-2021,webmail.sampo.ru,179624
2,17-Apr-2021,tvip-provision.sampo.ru,190335
3,17-Apr-2021,tvip-provision.sampo.ru,61670
4,17-Apr-2021,tvip-provision.sampo.ru,39370
...,...,...,...
10646532,01-Apr-2021,_sip._udp.048056.16.rt.ru,164026
10646533,01-Apr-2021,048056.16.rt.ru,133945
10646534,01-Apr-2021,camera.rt.ru,67986
10646535,01-Apr-2021,camera.rt.ru,31576


In [15]:
names['date'] = pd.to_datetime(names['date'])

In [16]:
names['number_request'] = names.groupby(names['contract_id'])['contract_id'].transform('size')
names = names.sort_values('date').drop_duplicates('contract_id', keep='last')
names
#тут идея точно такая же, как и с прошлыми данными

,date,url,contract_id,number_request
9826373,2021-03-26,cnt-odcv-itv03.svc.iptv.rt.ru,33897,4
9828862,2021-03-26,gg.rt.ru,50908,8
9829039,2021-03-26,wink.rt.ru,52503,1
9823571,2021-03-26,fnc.rt.ru,8040,2
9823515,2021-03-26,petrozavodsk.speedtest.rt.ru,3630,1
...,...,...,...,...
7271358,2021-04-30,acs.rt.ru,166854,58
7271357,2021-04-30,acs.rt.ru,127545,64
7271356,2021-04-30,camera.rt.ru,195055,61
7271428,2021-04-30,camera.rt.ru,88102,51


Итак, у нас есть датафреймы: train_data, contracts, logs, names.
Надо объединить их в одну большую таблицу по contract_id

In [17]:
final_data = contracts.merge(train_data, on='contract_id', how='outer')
final_data

,contract_id,day_or_month_contract,blocked
0,7780,0.0,0.0
1,3996,0.0,NaN
2,7785,0.0,0.0
3,7794,0.0,0.0
4,7795,1.0,0.0
...,...,...,...
6797,4261,0.0,NaN
6798,4118,1.0,NaN
6799,3843,1.0,NaN
6800,8188,NaN,0.0


In [18]:
data_to_predict = final_data[final_data['blocked'].isna()].reset_index().drop(columns=['index'])
data_to_predict
#делаем заготовку под список пользователей, для которых мы будем предсказывать отток после обучения

,contract_id,day_or_month_contract,blocked
0,3996,0.0,NaN
1,3469,0.0,NaN
2,3981,0.0,NaN
3,3860,0.0,NaN
4,3824,1.0,NaN
...,...,...,...
805,3503,1.0,NaN
806,3589,0.0,NaN
807,4261,0.0,NaN
808,4118,1.0,NaN


In [19]:
final_data = final_data[final_data['blocked'].notna()]
final_data = final_data.merge(logs, on='contract_id', how='outer')
final_data = final_data.merge(names, on='contract_id', how='outer')
final_data = final_data.reset_index().drop(columns=['index'])
#теперь оставляем только данные, для которых у нас известно значение целевого признака
#также подтягиваем данные из логов и обращений к конкурентам

cols_to_move = ['blocked']
new_cols = np.hstack((final_data.columns.difference(cols_to_move), cols_to_move))
final_data = final_data.loc[:, new_cols]
final_data
#Поместили целевой признак в последний столбец для удобства восприятия

,contract_id,date,day_or_month_contract,event_date,event_type,number_request,number_support,url,blocked
0,7780,2021-04-29,0.0,2021-04-28 15:12:24,Перенос денежных средств,83.0,9.0,iptv.sampo.ru,0.0
1,7785,2021-04-22,0.0,2021-03-30 18:21:41,Реестр договоров,10.0,1.0,fnc.rt.ru,0.0
2,7794,NaT,0.0,NaT,NaN,NaN,NaN,NaN,0.0
3,7795,NaT,1.0,2021-03-24 23:23:04,Включение интернета на 20 минут,NaN,7.0,NaN,0.0
4,7798,NaT,0.0,2021-04-17 20:07:11,Информер ВК. Показ,NaN,3.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...
199482,166854,2021-04-30,NaN,NaT,NaN,58.0,NaN,acs.rt.ru,NaN
199483,127545,2021-04-30,NaN,NaT,NaN,64.0,NaN,acs.rt.ru,NaN
199484,195055,2021-04-30,NaN,NaT,NaN,61.0,NaN,camera.rt.ru,NaN
199485,88102,2021-04-30,NaN,NaT,NaN,51.0,NaN,camera.rt.ru,NaN


In [20]:
final_data = final_data[final_data['blocked'].notna()]
final_data
#Это клиенты, по которым мы знаем итог и на которых мы можем обучать нейросеть

,contract_id,date,day_or_month_contract,event_date,event_type,number_request,number_support,url,blocked
0,7780,2021-04-29,0.0,2021-04-28 15:12:24,Перенос денежных средств,83.0,9.0,iptv.sampo.ru,0.0
1,7785,2021-04-22,0.0,2021-03-30 18:21:41,Реестр договоров,10.0,1.0,fnc.rt.ru,0.0
2,7794,NaT,0.0,NaT,NaN,NaN,NaN,NaN,0.0
3,7795,NaT,1.0,2021-03-24 23:23:04,Включение интернета на 20 минут,NaN,7.0,NaN,0.0
4,7798,NaT,0.0,2021-04-17 20:07:11,Информер ВК. Показ,NaN,3.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...
5987,274782,NaT,0.0,2021-04-22 13:04:56,Отключение услуги Автоплатёж,NaN,8.0,NaN,1.0
5988,274786,2021-04-05,1.0,2021-04-29 10:15:39,Обращение в службу заботы о клиентах,1.0,9.0,fnc.rt.ru,1.0
5989,274918,NaT,0.0,2021-04-15 13:06:10,Реестр договоров,NaN,1.0,NaN,1.0
5990,8188,NaT,NaN,NaT,NaN,NaN,NaN,NaN,0.0


In [21]:
final_data[['number_request', 'number_support']] = final_data[['number_request','number_support']].fillna(0)
#Тут предполагаем, что если у нас нет данных об обращений клиентов, то их и не было

/tmp/ipykernel_26635/708766708.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data[['number_request', 'number_support']] = final_data[['number_request','number_support']].fillna(0)


In [22]:
final_data[['event_type', 'url']] = final_data[['event_type','url']].fillna('No info')

/tmp/ipykernel_26635/4084625202.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data[['event_type', 'url']] = final_data[['event_type','url']].fillna('No info')


In [23]:
nodate = pd.to_datetime('1900-01-01')
final_data[['date', 'event_date']] =final_data[['date', 'event_date']].fillna(nodate)

/tmp/ipykernel_26635/2674313534.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data[['date', 'event_date']] =final_data[['date', 'event_date']].fillna(nodate)


In [24]:
final_data = final_data[final_data['day_or_month_contract'].notna()]
final_data
#таких клиентов всего 2, поэтому просто удалим их

,contract_id,date,day_or_month_contract,event_date,event_type,number_request,number_support,url,blocked
0,7780,2021-04-29,0.0,2021-04-28 15:12:24,Перенос денежных средств,83.0,9.0,iptv.sampo.ru,0.0
1,7785,2021-04-22,0.0,2021-03-30 18:21:41,Реестр договоров,10.0,1.0,fnc.rt.ru,0.0
2,7794,1900-01-01,0.0,1900-01-01 00:00:00,No info,0.0,0.0,No info,0.0
3,7795,1900-01-01,1.0,2021-03-24 23:23:04,Включение интернета на 20 минут,0.0,7.0,No info,0.0
4,7798,1900-01-01,0.0,2021-04-17 20:07:11,Информер ВК. Показ,0.0,3.0,No info,0.0
...,...,...,...,...,...,...,...,...,...
5985,274601,1900-01-01,1.0,2021-04-20 14:25:58,Информер ВК. Показ,0.0,6.0,No info,1.0
5986,274710,1900-01-01,0.0,2021-04-20 19:46:46,Информер ВК. Показ,0.0,7.0,No info,1.0
5987,274782,1900-01-01,0.0,2021-04-22 13:04:56,Отключение услуги Автоплатёж,0.0,8.0,No info,1.0
5988,274786,2021-04-05,1.0,2021-04-29 10:15:39,Обращение в службу заботы о клиентах,1.0,9.0,fnc.rt.ru,1.0


In [25]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5990 entries, 0 to 5989
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   contract_id            5990 non-null   int64         
 1   date                   5990 non-null   datetime64[ns]
 2   day_or_month_contract  5990 non-null   float64       
 3   event_date             5990 non-null   datetime64[ns]
 4   event_type             5990 non-null   object        
 5   number_request         5990 non-null   float64       
 6   number_support         5990 non-null   float64       
 7   url                    5990 non-null   object        
 8   blocked                5990 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 468.0+ KB


In [26]:
X = final_data.drop(columns=['blocked'])
y = final_data.loc[:, ['blocked']]

In [27]:
X['date_year'] = X['date'].dt.year
X['date_month'] = X['date'].dt.month
X['date_day'] = X['date'].dt.day

X['event_date_year'] = X['event_date'].dt.year
X['event_date_month'] = X['event_date'].dt.month
X['event_date_day'] = X['event_date'].dt.day

#разбиваем дату на три числовых признака

In [28]:
X = X.drop(columns=['date','event_date'])

In [29]:
def get_cat_indicies(X):
    cats = []
    for col in X.columns:
        if is_numeric_dtype(X[col]):
            pass
        else:
            cats.append(col)
    cat_indicies = []
    for col in cats:
        cat_indicies.append(X.columns.get_loc(col))
    return cat_indicies
categorical_indicies = get_cat_indicies(X)

In [30]:
def convert_cats(X):
    cats = []
    for col in X.columns:
        if is_numeric_dtype(X[col]):
            pass
        else:
            cats.append(col)
    cat_indicies = []
    for col in cats:
        X[col] = X[col].astype('category')
convert_cats(X)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101, stratify=y)

In [32]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4792 entries, 4541 to 4974
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   contract_id            4792 non-null   int64   
 1   day_or_month_contract  4792 non-null   float64 
 2   event_type             4792 non-null   category
 3   number_request         4792 non-null   float64 
 4   number_support         4792 non-null   float64 
 5   url                    4792 non-null   category
 6   date_year              4792 non-null   int64   
 7   date_month             4792 non-null   int64   
 8   date_day               4792 non-null   int64   
 9   event_date_year        4792 non-null   int64   
 10  event_date_month       4792 non-null   int64   
 11  event_date_day         4792 non-null   int64   
dtypes: category(2), float64(3), int64(7)
memory usage: 434.0 KB


In [33]:
train_dataset = cb.Pool(X_train,y_train, cat_features=categorical_indicies)
test_dataset = cb.Pool(X_test,y_test, cat_features=categorical_indicies)

In [34]:
model = cb.CatBoostClassifier(loss_function='Logloss', eval_metric='Accuracy', nan_mode='Min')

In [35]:
grid = {'learning_rate': [0.03, 0.1],
'depth': [4, 6, 10],
'l2_leaf_reg': [1, 3, 5,],
'iterations': [50, 100, 150]}

In [36]:
model.grid_search(grid,train_dataset)

0:	learn: 0.9822593	test: 0.9885297	best: 0.9885297 (0)	total: 53.4ms	remaining: 2.62s
1:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 57.7ms	remaining: 1.38s
2:	learn: 0.9872163	test: 0.9916580	best: 0.9937435 (1)	total: 61.7ms	remaining: 967ms
3:	learn: 0.9851291	test: 0.9895725	best: 0.9937435 (1)	total: 65.5ms	remaining: 753ms
4:	learn: 0.9874772	test: 0.9916580	best: 0.9937435 (1)	total: 69.4ms	remaining: 624ms
5:	learn: 0.9859118	test: 0.9906152	best: 0.9937435 (1)	total: 73.1ms	remaining: 536ms
6:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 76.8ms	remaining: 472ms
7:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 79.6ms	remaining: 418ms
8:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 82.3ms	remaining: 375ms
9:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 84.8ms	remaining: 339ms
10:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 87.1ms	remaining: 309ms
11:	learn: 0.9877381	test: 0.9937435	best:

24:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 51ms	remaining: 51ms
25:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 54.1ms	remaining: 49.9ms
26:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 56.7ms	remaining: 48.3ms
27:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 59.8ms	remaining: 47ms
28:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 62.6ms	remaining: 45.4ms
29:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 66.5ms	remaining: 44.4ms
30:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 69.6ms	remaining: 42.6ms
31:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 72.3ms	remaining: 40.6ms
32:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 75.1ms	remaining: 38.7ms
33:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 78ms	remaining: 36.7ms
34:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 80.6ms	remaining: 34.5ms
35:	learn: 0.9877381	test: 0.

9:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 22ms	remaining: 87.9ms
10:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 24.7ms	remaining: 87.5ms
11:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 27.5ms	remaining: 87ms
12:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 30ms	remaining: 85.4ms
13:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 32.8ms	remaining: 84.2ms
14:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 34.6ms	remaining: 80.7ms
15:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 36.4ms	remaining: 77.5ms
16:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 38.2ms	remaining: 74.2ms
17:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 40ms	remaining: 71.2ms
18:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 41.9ms	remaining: 68.4ms
19:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 43.7ms	remaining: 65.6ms
20:	learn: 0.9877381	test: 0.9

63:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 134ms	remaining: 75.2ms
64:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 136ms	remaining: 73.2ms
65:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 138ms	remaining: 71ms
66:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 140ms	remaining: 68.9ms
67:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 142ms	remaining: 66.7ms
68:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 144ms	remaining: 64.5ms
69:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 145ms	remaining: 62.3ms
70:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 147ms	remaining: 60.1ms
71:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 150ms	remaining: 58.1ms
72:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 152ms	remaining: 56ms
73:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (1)	total: 153ms	remaining: 53.9ms
74:	learn: 0.9877381	test: 0.9937435

64:	learn: 0.9879990	test: 0.9937435	best: 0.9937435 (1)	total: 129ms	remaining: 69.7ms
65:	learn: 0.9879990	test: 0.9937435	best: 0.9937435 (1)	total: 132ms	remaining: 68ms
66:	learn: 0.9879990	test: 0.9937435	best: 0.9937435 (1)	total: 135ms	remaining: 66.6ms
67:	learn: 0.9882598	test: 0.9937435	best: 0.9937435 (1)	total: 137ms	remaining: 64.6ms
68:	learn: 0.9882598	test: 0.9937435	best: 0.9937435 (1)	total: 139ms	remaining: 62.4ms
69:	learn: 0.9882598	test: 0.9937435	best: 0.9937435 (1)	total: 141ms	remaining: 60.4ms
70:	learn: 0.9882598	test: 0.9937435	best: 0.9937435 (1)	total: 143ms	remaining: 58.2ms
71:	learn: 0.9885207	test: 0.9937435	best: 0.9937435 (1)	total: 144ms	remaining: 56.1ms
72:	learn: 0.9882598	test: 0.9937435	best: 0.9937435 (1)	total: 146ms	remaining: 54ms
73:	learn: 0.9885207	test: 0.9937435	best: 0.9937435 (1)	total: 148ms	remaining: 52ms
74:	learn: 0.9885207	test: 0.9937435	best: 0.9937435 (1)	total: 150ms	remaining: 49.9ms
75:	learn: 0.9885207	test: 0.9937435	b

...

98:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (2)	total: 487ms	remaining: 251ms
99:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (2)	total: 504ms	remaining: 252ms
100:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (2)	total: 510ms	remaining: 247ms
101:	learn: 0.9877381	test: 0.9937435	best: 0.9937435 (2)	total: 516ms	remaining: 243ms
102:	learn: 0.9879990	test: 0.9937435	best: 0.9937435 (2)	total: 522ms	remaining: 238ms
103:	learn: 0.9879990	test: 0.9937435	best: 0.9937435 (2)	total: 528ms	remaining: 234ms
104:	learn: 0.9879990	test: 0.9937435	best: 0.9937435 (2)	total: 534ms	remaining: 229ms
105:	learn: 0.9879990	test: 0.9937435	best: 0.9937435 (2)	total: 540ms	remaining: 224ms
106:	learn: 0.9879990	test: 0.9937435	best: 0.9937435 (2)	total: 545ms	remaining: 219ms
107:	learn: 0.9879990	test: 0.9937435	best: 0.9937435 (2)	total: 551ms	remaining: 214ms
108:	learn: 0.9879990	test: 0.9937435	best: 0.9937435 (2)	total: 557ms	remaining: 209ms
109:	learn: 0.9879990	test: 0.9937

57:	learn: 0.9906079	test: 0.9937435	best: 0.9937435 (2)	total: 303ms	remaining: 481ms
58:	learn: 0.9906079	test: 0.9937435	best: 0.9937435 (2)	total: 314ms	remaining: 485ms
59:	learn: 0.9906079	test: 0.9937435	best: 0.9937435 (2)	total: 321ms	remaining: 481ms
60:	learn: 0.9906079	test: 0.9937435	best: 0.9937435 (2)	total: 326ms	remaining: 476ms
61:	learn: 0.9906079	test: 0.9937435	best: 0.9937435 (2)	total: 329ms	remaining: 466ms
62:	learn: 0.9906079	test: 0.9937435	best: 0.9937435 (2)	total: 333ms	remaining: 459ms
63:	learn: 0.9906079	test: 0.9937435	best: 0.9937435 (2)	total: 339ms	remaining: 455ms
64:	learn: 0.9906079	test: 0.9937435	best: 0.9937435 (2)	total: 341ms	remaining: 447ms
65:	learn: 0.9906079	test: 0.9937435	best: 0.9937435 (2)	total: 348ms	remaining: 443ms
66:	learn: 0.9906079	test: 0.9937435	best: 0.9937435 (2)	total: 351ms	remaining: 434ms
67:	learn: 0.9906079	test: 0.9937435	best: 0.9937435 (2)	total: 353ms	remaining: 426ms
68:	learn: 0.9906079	test: 0.9937435	best: 

19:	learn: 0.9893550	test: 0.9881101	best: 0.9881101 (4)	total: 87.7ms	remaining: 132ms
20:	learn: 0.9896681	test: 0.9881101	best: 0.9881101 (4)	total: 93.5ms	remaining: 129ms
21:	learn: 0.9899812	test: 0.9881101	best: 0.9881101 (4)	total: 99.1ms	remaining: 126ms
22:	learn: 0.9902943	test: 0.9881101	best: 0.9881101 (4)	total: 105ms	remaining: 123ms
23:	learn: 0.9902943	test: 0.9881101	best: 0.9881101 (4)	total: 111ms	remaining: 120ms
24:	learn: 0.9906074	test: 0.9881101	best: 0.9881101 (4)	total: 117ms	remaining: 117ms
25:	learn: 0.9915466	test: 0.9881101	best: 0.9881101 (4)	total: 122ms	remaining: 113ms
26:	learn: 0.9918597	test: 0.9881101	best: 0.9881101 (4)	total: 127ms	remaining: 108ms
27:	learn: 0.9918597	test: 0.9881101	best: 0.9881101 (4)	total: 132ms	remaining: 104ms
28:	learn: 0.9918597	test: 0.9881101	best: 0.9881101 (4)	total: 138ms	remaining: 100ms
29:	learn: 0.9918597	test: 0.9881101	best: 0.9881101 (4)	total: 145ms	remaining: 96.4ms
30:	learn: 0.9918597	test: 0.9881101	be

43:	learn: 0.9918623	test: 0.9899812	best: 0.9906074 (14)	total: 207ms	remaining: 28.3ms
44:	learn: 0.9918623	test: 0.9899812	best: 0.9906074 (14)	total: 214ms	remaining: 23.8ms
45:	learn: 0.9921753	test: 0.9899812	best: 0.9906074 (14)	total: 221ms	remaining: 19.3ms
46:	learn: 0.9921753	test: 0.9899812	best: 0.9906074 (14)	total: 228ms	remaining: 14.5ms
47:	learn: 0.9921753	test: 0.9899812	best: 0.9906074 (14)	total: 230ms	remaining: 9.57ms
48:	learn: 0.9921753	test: 0.9899812	best: 0.9906074 (14)	total: 237ms	remaining: 4.83ms
49:	learn: 0.9928013	test: 0.9899812	best: 0.9906074 (14)	total: 243ms	remaining: 0us

bestTest = 0.9906073889
bestIteration = 14



{'params': {'depth': 10,
  'l2_leaf_reg': 1,
  'iterations': 50,
  'learning_rate': 0.1},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,

In [37]:
model.get_params()

{'loss_function': 'Logloss',
 'nan_mode': 'Min',
 'eval_metric': 'Accuracy',
 'depth': 10,
 'l2_leaf_reg': 1,
 'iterations': 50,
 'learning_rate': 0.1}

In [38]:
pred = model.predict(X_test)

In [39]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      1058
         1.0       0.99      0.89      0.94       140

    accuracy                           0.99      1198
   macro avg       0.99      0.95      0.97      1198
weighted avg       0.99      0.99      0.99      1198



In [40]:
data_to_predict = data_to_predict.merge(logs, on='contract_id', how='left')
data_to_predict = data_to_predict.merge(names, on='contract_id', how='left')
data_to_predict[['number_request', 'number_support']] = data_to_predict[['number_request','number_support']].fillna(value=0)
data_to_predict[['event_type', 'url']] = data_to_predict[['event_type','url']].fillna('No info')
nodate = pd.to_datetime('1900-01-01')
data_to_predict[['date', 'event_date']] =data_to_predict[['date', 'event_date']].fillna(nodate)
data_to_predict

,contract_id,day_or_month_contract,blocked,event_date,event_type,number_support,date,url,number_request
0,3996,0.0,NaN,1900-01-01 00:00:00,No info,0.0,1900-01-01,No info,0.0
1,3469,0.0,NaN,2021-04-13 16:51:49,Обращение в службу заботы о клиентах,9.0,2021-04-22,fnc.rt.ru,3.0
2,3981,0.0,NaN,2021-03-10 16:20:49,Состояние клиентского оборудования,3.0,2021-04-23,data.sampo.ru,4.0
3,3860,0.0,NaN,1900-01-01 00:00:00,No info,0.0,1900-01-01,No info,0.0
4,3824,1.0,NaN,1900-01-01 00:00:00,No info,0.0,1900-01-01,No info,0.0
...,...,...,...,...,...,...,...,...,...
805,3503,1.0,NaN,2021-04-14 16:49:28,Обращение в службу заботы о клиентах,12.0,1900-01-01,No info,0.0
806,3589,0.0,NaN,2021-03-29 13:23:26,Чек лист выполненных работ при регистрации,5.0,2021-04-09,fnc.rt.ru,7.0
807,4261,0.0,NaN,2021-03-31 12:09:15,Обращение в службу заботы о клиентах,18.0,1900-01-01,No info,0.0
808,4118,1.0,NaN,2021-04-27 23:44:49,Виртуальное бессмертие. Использование,19.0,2021-04-07,fnc.rt.ru,3.0


In [41]:
data_to_predict['date_year'] = data_to_predict['date'].dt.year
data_to_predict['date_month'] = data_to_predict['date'].dt.month
data_to_predict['date_day'] = data_to_predict['date'].dt.day

data_to_predict['event_date_year'] = data_to_predict['event_date'].dt.year
data_to_predict['event_date_month'] = data_to_predict['event_date'].dt.month
data_to_predict['event_date_day'] = data_to_predict['event_date'].dt.day

In [42]:
cols_to_move = ['blocked']
new_cols = np.hstack((data_to_predict.columns.difference(cols_to_move), cols_to_move))
data_to_predict = data_to_predict.loc[:, new_cols]
data_to_predict = data_to_predict.drop(columns=['blocked','date','event_date'])

In [43]:
data_to_predict=data_to_predict.reindex(columns=list(X_test.columns))
print(data_to_predict.info())
print(X_test.info())
#делаем порядок колонок одинаковым

<class 'pandas.core.frame.DataFrame'>
Int64Index: 810 entries, 0 to 809
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   contract_id            810 non-null    int64  
 1   day_or_month_contract  810 non-null    float64
 2   event_type             810 non-null    object 
 3   number_request         810 non-null    float64
 4   number_support         810 non-null    float64
 5   url                    810 non-null    object 
 6   date_year              810 non-null    int64  
 7   date_month             810 non-null    int64  
 8   date_day               810 non-null    int64  
 9   event_date_year        810 non-null    int64  
 10  event_date_month       810 non-null    int64  
 11  event_date_day         810 non-null    int64  
dtypes: float64(3), int64(7), object(2)
memory usage: 82.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1198 entries, 2355 to 5597
Data columns (total 12

In [44]:
convert_cats(data_to_predict)
#конвертируем категориальные признаки

In [45]:
answer = model.predict(data_to_predict)

In [46]:
data_to_predict['blocked'] = answer

In [47]:
data_to_predict

,contract_id,day_or_month_contract,event_type,number_request,number_support,url,date_year,date_month,date_day,event_date_year,event_date_month,event_date_day,blocked
0,3996,0.0,No info,0.0,0.0,No info,1900,1,1,1900,1,1,0.0
1,3469,0.0,Обращение в службу заботы о клиентах,3.0,9.0,fnc.rt.ru,2021,4,22,2021,4,13,0.0
2,3981,0.0,Состояние клиентского оборудования,4.0,3.0,data.sampo.ru,2021,4,23,2021,3,10,0.0
3,3860,0.0,No info,0.0,0.0,No info,1900,1,1,1900,1,1,0.0
4,3824,1.0,No info,0.0,0.0,No info,1900,1,1,1900,1,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,3503,1.0,Обращение в службу заботы о клиентах,0.0,12.0,No info,1900,1,1,2021,4,14,0.0
806,3589,0.0,Чек лист выполненных работ при регистрации,7.0,5.0,fnc.rt.ru,2021,4,9,2021,3,29,0.0
807,4261,0.0,Обращение в службу заботы о клиентах,0.0,18.0,No info,1900,1,1,2021,3,31,0.0
808,4118,1.0,Виртуальное бессмертие. Использование,3.0,19.0,fnc.rt.ru,2021,4,7,2021,4,27,0.0


In [48]:
data_to_predict.to_csv('solution.csv', columns = ['contract_id','blocked'])